In [97]:
from bs4 import BeautifulSoup
from playwright.async_api import async_playwright
from renetti.ws.spiders.utils import parse_product_json_ld_from_page
import urllib.parse

In [98]:
url = "https://eleiko.com/en-gb/equipment/benches/competitionbenches"
base_url = "https://eleiko.com/"

In [21]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                await page.goto(url=url)
                await page.wait_for_timeout(2000)
                html = await page.content()
                soup = BeautifulSoup(markup=html, features="html.parser")
                urls = [
                    f"{base_url}{a.get("href")}" for art in soup.find_all("article") for a in art.find_all("a")
                ]

Page.wait_for_timeout: Target page, context or browser has been closed


In [94]:
url = "https://eleiko.com/en-gb/equipment/benches/competitionbenches/3085245-060-eleiko-ipf-competition-combo-rack-pur-charcoal"
url = "https://eleiko.com/en-gb/equipment/plates/fitness/3085125-20-eleiko-xf-bumper-plate-20-kg-black"

In [95]:
async with async_playwright() as playwright:
    async with await playwright.chromium.launch(headless=False) as browser:
        async with await browser.new_context() as context:
            async with await context.new_page() as page:
                await page.goto(url=url)
                html = await page.content()
                soup = BeautifulSoup(markup=html)
                scraped_equipment = parse_product_json_ld_from_page(soup=soup)
                image_links = []
                for img in soup.findAll("img", class_="lg:group-hover:scale-103"):
                    raw_img_url = img.get("src").split("url=")[1]
                    img_url = urllib.parse.unquote(raw_img_url)
                    img_url = img_url.split("&")[0]
                    image_links.append(img_url)

                categories = url.split("/equipment")[1].split("/")[1:-1]
                description = soup.find_all("p", class_="lg:leading-normal")[0].text

                scraped_equipment["brands"] = ["eleiko"]
                scraped_equipment["image_links"] = image_links
                scraped_equipment["categories"] = categories
                scraped_equipment["description"] = description

In [96]:
scraped_equipment

{'name': 'Eleiko XF Bumper Plate - 20 kg, black',
 'image_links': ['https://media.eleiko.com/images/upload/4x5/3085125-20_10.jpg',
  'https://media.eleiko.com/images/upload/4x5/3085125-20_20.jpg',
  'https://media.eleiko.com/images/upload/4x5/3085125-20_30.jpg',
  'https://media.eleiko.com/images/upload/4x5/3085125-20_40.jpg'],
 'mpn': None,
 'description': 'The new Eleiko XF Bumpers feature an improved composition for enhanced durability. They remain an incredibly quiet disc designed for tough training and varied environments but now will last even longer. We have also created a more narrow 20 kg discs, one that matches the width of the 15 kg discs, so more weight can be loaded on the barbell, now up to 200 kg. Our compressed recycled crumb rubber is soft yet durable providing fantastic sound dampening and shock absorption. Discs are sold individually.',
 'brands': ['eleiko'],
 'categories': ['plates', 'fitness'],
 'skus': []}